In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tweepy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tweepy
    Found existing installation: tweepy 4.13.0
    Uninstalling tweepy-4.13.0:
      Successfully uninstalled tweepy-4.13.0


In [ ]:
import tweepy 

# https://docs.tweepy.org/en/stable/examples.html

client = tweepy.Client(bearer_token='bearer_token', wait_on_rate_limit=True)

#Topics used for the project are: war in ukraine, earthquake in turkey, climate change, overpopulation and veganism   64k of data collected
with open('/content/drive/MyDrive/data2.txt', 'a', encoding="utf-8") as txt:
    for tweet in tweepy.Paginator(client.search_recent_tweets, query='veganism -is:retweet',max_results=100).flatten(limit=6400):     
        txt.write(str(tweet.text) + '\n')

In [4]:
import pandas as pd
import re 
import spacy
import numpy as np
from tqdm import tqdm

In [10]:
nlp=spacy.load("en_core_web_sm")
# for token in nlp("She ate the pizza"):
#   print(token.text,token.pos_)
nlp.add_pipe('sentencizer')

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/test2.txt',error_bad_lines=False,names=["Tweets"])
#blank lines removed data shrunk to 37k
df.to_csv('/content/drive/MyDrive/test10_csv.csv')

In [7]:
df.sample(5)

,Tweets
10285,@5ireChain amazes me with how they approach cl...
1656,https://t.co/HdwIm4cyKP
32757,@brettbergie Climate change causes arson.
320,#Canada #Germany #bryansk #Ukraine #Bakhmut
6940,Russia-Ukraine War Live:


In [8]:
from re import sub

def drop_short(text):
    text_len = len(text.split())
    if text_len > 2:
        return text
    else:
        return ''

def remove_stopwords(text):
    return " ".join(filter(lambda x: x not in nlp.Defaults.stop_words, text.split()))

def lemmatize(text):
    return " ".join([x.lemma_ for x in nlp(text)])

def regex(tweets): 

  # remove user handles
  tweets = sub(r'@[^\s]+', " ", tweets)

  # remove links
  tweets = sub(r'((www.[^\s]+)|(https?://[^\s]+))', " ", tweets)

  # remove special characters
  tweets = sub(r"[^a-zA-Z\s']", " ", tweets)

  # remove tweets shorter than 2 words
  #tweets = sub(r"\s{2,}", " ", tweets)

  return tweets

In [11]:
df = pd.read_csv('/content/drive/MyDrive/test10_csv.csv')
tweets = df["Tweets"]

# change uppercase characters
tweets = tweets.apply(lambda x: x.lower())

tweets = tweets.apply(lambda x: regex(x))

# remove tweets shorter than 2 words
tweets = tweets.apply(lambda x: drop_short(x))

# remove stop words
tweets = tweets.apply(remove_stopwords)

# lemmatize
tweets = tweets.apply(lemmatize)

# remove nulls 
tweets.replace('', np.nan, inplace=True)
tweets.dropna(inplace=True)

# remove duplicates
tweets.drop_duplicates(keep="first", inplace=True)

# 21k of clean data
tweets.reset_index(drop=True, inplace=True)
tweets.to_csv('/content/drive/MyDrive/clean17.csv', index=False)

In [12]:
df = pd.read_csv('/content/drive/MyDrive/clean17.csv')
df.sample(5)

,Tweets
7566,republican policy prevent bodily autonomy prot...
9331,earth org
12099,jenny rissved beslut har b rjat ta k tt igen
13133,greatness nation moral progress judge way anim...
8922,attend informative wonderful talk session huma...
